## Basic information
#### frame of thickness is 0.084 mm


#### 读取TIFF图像堆栈

In [4]:
from PIL import Image
import numpy as np

# 读取多帧 TIFF
img = Image.open("/Users/harrysong/Desktop/agriculture/tiff/020150.tiff")

frames = []
try:
    while True:
        frame = np.array(img.convert("L"))  # 灰度图
        frames.append(frame)
        img.seek(img.tell() + 1)
except EOFError:
    pass  # 到达文件末尾

print(f"共 {len(frames)} 帧")


共 2760 帧


#### 在每帧中检测圆形（麦秆）

In [5]:
import cv2

def detect_stalk_circle(frame):
    blurred = cv2.medianBlur(frame, 5)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2,
                               minDist=30, param1=50, param2=30,
                               minRadius=10, maxRadius=100)
    if circles is not None:
        return True
    return False


#### 找出麦秆开始和结束的帧

In [6]:
start_frame = None
end_frame = None

for i, frame in enumerate(frames):
    if detect_stalk_circle(frame):
        if start_frame is None:
            start_frame = i
        end_frame = i  # 更新最后检测到的帧

if start_frame is not None and end_frame is not None:
    print(f"麦秆从第 {start_frame} 帧到第 {end_frame} 帧")


麦秆从第 766 帧到第 2573 帧
